In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import os

#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
cuda0 = torch.device('cuda:0')
x = torch.tensor([1., 2.], device=cuda0)
print(x)

tt=torch.cuda.current_device()
print(tt)
torch.cuda.device_count()

b = torch.tensor([1., 2.]).cuda()
print(b)

torch.cuda.get_device_name()


tensor([1., 2.], device='cuda:0')
0
tensor([1., 2.], device='cuda:0')


'NVIDIA GeForce RTX 3060'

In [3]:
#c = torch.tensor([1., 2.])
#print(c)
#cc = c.to(cuda0)
#print(cc)

In [4]:
#torch.cuda.get_device_properties(0)


In [5]:
input_size = 784       # Размеры изображения = 28 x 28 = 784
hidden_size = 500      # Количество узлов на скрытом слое
num_classes = 10       # Число классов на выходе. В этом случае от 0 до 9
num_epochs = 5         # Количество тренировок всего набора данных
batch_size = 100       # Размер входных данных для одной итерации
learning_rate = 0.001  # Скорость конвергенции





In [6]:
train_dataset = dsets.MNIST(
    root='./MNIST',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
 
test_dataset = dsets.MNIST(
    root='./MNIST',
    train=False,
    transform=transforms.ToTensor()
)

In [7]:
print(type(train_dataset))

<class 'torchvision.datasets.mnist.MNIST'>


In [8]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
 
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [9]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Наследуемый родительским классом nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1й связанный слой: 784 (данные входа) -> 500 (скрытый узел)
        self.relu = nn.ReLU()                          # Нелинейный слой ReLU max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2й связанный слой: 500 (скрытый узел) -> 10 (класс вывода)
    
    def forward(self, x):                              # Передний пропуск: складывание каждого слоя вместе
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [10]:
net = Net(input_size, hidden_size, num_classes)
device = torch.device(cuda0)

In [11]:
net.cuda() # Включаем GPU

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [12]:
print(net.state_dict())

OrderedDict([('fc1.weight', tensor([[-1.7886e-02,  2.4059e-02,  1.2127e-02,  ...,  8.2212e-03,
         -1.6209e-02, -2.3337e-02],
        [-3.1160e-02, -2.8851e-02, -2.9725e-02,  ...,  8.0366e-05,
          2.5447e-02,  1.7749e-02],
        [ 2.4921e-02, -1.5488e-02, -1.4563e-02,  ..., -3.3916e-02,
         -3.5218e-02,  4.6975e-03],
        ...,
        [-6.3645e-04,  1.0396e-02, -1.9093e-02,  ...,  2.0065e-02,
          8.5642e-03,  2.7956e-02],
        [-3.0655e-02,  2.6850e-02, -2.5427e-02,  ..., -1.5808e-02,
          3.0509e-02, -2.7482e-02],
        [ 2.1631e-03, -5.4711e-03, -3.0814e-02,  ...,  3.2413e-02,
          1.8580e-02, -1.5256e-02]], device='cuda:0')), ('fc1.bias', tensor([ 2.6183e-02,  3.0136e-02, -2.7451e-02, -2.9345e-02, -6.3442e-03,
        -1.4809e-02, -3.3506e-02,  3.3064e-02, -9.8210e-03, -1.1015e-02,
         3.2388e-02,  2.4688e-02,  6.9772e-03,  1.4238e-03, -6.5163e-03,
         3.5321e-02,  2.9798e-02,  2.1694e-02,  1.8111e-02, -2.9808e-02,
         4.7781e

In [13]:
criterion = nn.CrossEntropyLoss()   # Функция потерь
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)   # Оптимизатор градиентного спуска

In [14]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Загрузка партии изображений с индексом, данными, классом
             
        images = Variable(images.view(-1, 28*28)).to(cuda0)         # Конвертация тензора в переменную: изменяем изображение с вектора, размером 784 на матрицу 28 x 28
        labels = Variable(labels).to(cuda0)
        
        optimizer.zero_grad()                             # Инициализация скрытых масс до нулей
        outputs = net(images)     # Передний пропуск: определение выходного класса, данного изображения
            
        loss = criterion(outputs, labels)                # Определение потерь: разница между выходным классом и предварительно заданной меткой
        loss.backward()                                   # Обратный проход: определение параметра weight
        optimizer.step()                                  # Оптимизатор: обновление параметров веса в скрытых узлах
        
        if (i+1) % 100 == 0:                              # Логирование
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data.item()))
            

Epoch [1/5], Step [100/600], Loss: 0.4704
Epoch [1/5], Step [200/600], Loss: 0.4836
Epoch [1/5], Step [300/600], Loss: 0.2666
Epoch [1/5], Step [400/600], Loss: 0.2918
Epoch [1/5], Step [500/600], Loss: 0.1363
Epoch [1/5], Step [600/600], Loss: 0.0959
Epoch [2/5], Step [100/600], Loss: 0.2181
Epoch [2/5], Step [200/600], Loss: 0.1358
Epoch [2/5], Step [300/600], Loss: 0.0681
Epoch [2/5], Step [400/600], Loss: 0.0967
Epoch [2/5], Step [500/600], Loss: 0.1020
Epoch [2/5], Step [600/600], Loss: 0.1054
Epoch [3/5], Step [100/600], Loss: 0.0979
Epoch [3/5], Step [200/600], Loss: 0.1545
Epoch [3/5], Step [300/600], Loss: 0.0497
Epoch [3/5], Step [400/600], Loss: 0.1606
Epoch [3/5], Step [500/600], Loss: 0.0590
Epoch [3/5], Step [600/600], Loss: 0.0386
Epoch [4/5], Step [100/600], Loss: 0.0338
Epoch [4/5], Step [200/600], Loss: 0.0306
Epoch [4/5], Step [300/600], Loss: 0.0580
Epoch [4/5], Step [400/600], Loss: 0.0351
Epoch [4/5], Step [500/600], Loss: 0.0308
Epoch [4/5], Step [600/600], Loss:

In [15]:
torch.save(net.state_dict(), 'fnn_model.pkl')

In [16]:


print(net) # Структура сети

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [17]:
# Просмотр параметров (веса и смещения)

for name, param in net.named_parameters():
    print('name: ', name)
    print(type(param))
    print('param.shape: ', param.shape)
    print('param.requires_grad: ', param.requires_grad)
    print('=====')

name:  fc1.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:  torch.Size([500, 784])
param.requires_grad:  True
=====
name:  fc1.bias
<class 'torch.nn.parameter.Parameter'>
param.shape:  torch.Size([500])
param.requires_grad:  True
=====
name:  fc2.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:  torch.Size([10, 500])
param.requires_grad:  True
=====
name:  fc2.bias
<class 'torch.nn.parameter.Parameter'>
param.shape:  torch.Size([10])
param.requires_grad:  True
=====


In [18]:

for name, param in net.named_parameters():
    if name in ['fc2.weight', 'fc2.bias']:
        param.requires_grad = True
    else:
        param.requires_grad = False

for name, param in net.named_parameters():
    print(name, ':', param.requires_grad)        

fc1.weight : False
fc1.bias : False
fc2.weight : True
fc2.bias : True


In [19]:
#сохраняем веса

torch.save (net.state_dict(), "weights_path_name.pth")

In [20]:
# сохраняем всю модель

torch.save (net.state_dict(), "model.pth")


In [21]:
print(net.state_dict())

OrderedDict([('fc1.weight', tensor([[-1.7886e-02,  2.4059e-02,  1.2127e-02,  ...,  8.2212e-03,
         -1.6209e-02, -2.3337e-02],
        [-3.1160e-02, -2.8851e-02, -2.9725e-02,  ...,  8.0366e-05,
          2.5447e-02,  1.7749e-02],
        [ 2.4921e-02, -1.5488e-02, -1.4563e-02,  ..., -3.3916e-02,
         -3.5218e-02,  4.6975e-03],
        ...,
        [-6.3645e-04,  1.0396e-02, -1.9093e-02,  ...,  2.0065e-02,
          8.5642e-03,  2.7956e-02],
        [-3.0655e-02,  2.6850e-02, -2.5427e-02,  ..., -1.5808e-02,
          3.0509e-02, -2.7482e-02],
        [ 2.1631e-03, -5.4711e-03, -3.0814e-02,  ...,  3.2413e-02,
          1.8580e-02, -1.5256e-02]], device='cuda:0')), ('fc1.bias', tensor([ 8.2043e-02,  6.0930e-02,  9.9372e-03, -5.6765e-02,  6.8273e-02,
         1.3366e-02, -2.4556e-02,  5.1160e-02, -5.6142e-02, -1.2249e-02,
        -5.8979e-03,  8.6617e-02,  1.0421e-01,  3.0909e-02,  4.6933e-02,
         8.8124e-02,  7.6432e-02,  7.6576e-02,  6.7188e-02, -2.8336e-02,
         3.3370e

In [22]:
model_new = Net(input_size, hidden_size, num_classes)
device = torch.device(cuda0)
model_new.cuda()

model_new.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [23]:
for name, param in model_new.named_parameters():
    print(name, ':', param.requires_grad)

fc1.weight : True
fc1.bias : True
fc2.weight : True
fc2.bias : True


In [24]:
torch.save(net, 'entire_model.pth')

In [25]:
model_new1 = torch.load('entire_model.pth')

In [26]:
for name, param in model_new1.named_parameters():
    print(name, ':', param.requires_grad)

fc1.weight : False
fc1.bias : False
fc2.weight : True
fc2.bias : True


In [27]:
device = torch.device(cuda0)
model_new1.cuda(0)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [28]:
criterion = nn.CrossEntropyLoss()   # Функция потерь
optimizer = torch.optim.Adam(model_new1.parameters(), lr=learning_rate)   # Оптимизатор градиентного спуска

In [29]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Загрузка партии изображений с индексом, данными, классом
             
        images = Variable(images.view(-1, 28*28)).to(cuda0)         # Конвертация тензора в переменную: изменяем изображение с вектора, размером 784 на матрицу 28 x 28
        labels = Variable(labels).to(cuda0)
        
        optimizer.zero_grad()                             # Инициализация скрытых масс до нулей
        outputs = model_new1(images)     # Передний пропуск: определение выходного класса, данного изображения
            
        loss = criterion(outputs, labels)                # Определение потерь: разница между выходным классом и предварительно заданной меткой
        loss.backward()                                   # Обратный проход: определение параметра weight
        optimizer.step()                                  # Оптимизатор: обновление параметров веса в скрытых узлах
        
        if (i+1) % 100 == 0:                              # Логирование
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data.item()))

Epoch [1/5], Step [100/600], Loss: 0.0071
Epoch [1/5], Step [200/600], Loss: 0.0144
Epoch [1/5], Step [300/600], Loss: 0.0103
Epoch [1/5], Step [400/600], Loss: 0.0339
Epoch [1/5], Step [500/600], Loss: 0.0061
Epoch [1/5], Step [600/600], Loss: 0.0364
Epoch [2/5], Step [100/600], Loss: 0.0281
Epoch [2/5], Step [200/600], Loss: 0.0195
Epoch [2/5], Step [300/600], Loss: 0.0067
Epoch [2/5], Step [400/600], Loss: 0.0326
Epoch [2/5], Step [500/600], Loss: 0.0293
Epoch [2/5], Step [600/600], Loss: 0.0088
Epoch [3/5], Step [100/600], Loss: 0.0337
Epoch [3/5], Step [200/600], Loss: 0.0051
Epoch [3/5], Step [300/600], Loss: 0.0061
Epoch [3/5], Step [400/600], Loss: 0.0015
Epoch [3/5], Step [500/600], Loss: 0.0032
Epoch [3/5], Step [600/600], Loss: 0.0046
Epoch [4/5], Step [100/600], Loss: 0.0216
Epoch [4/5], Step [200/600], Loss: 0.0007
Epoch [4/5], Step [300/600], Loss: 0.0168
Epoch [4/5], Step [400/600], Loss: 0.0052
Epoch [4/5], Step [500/600], Loss: 0.0101
Epoch [4/5], Step [600/600], Loss:

In [30]:
print(model_new1.state_dict())

OrderedDict([('fc1.weight', tensor([[-1.7886e-02,  2.4059e-02,  1.2127e-02,  ...,  8.2212e-03,
         -1.6209e-02, -2.3337e-02],
        [-3.1160e-02, -2.8851e-02, -2.9725e-02,  ...,  8.0366e-05,
          2.5447e-02,  1.7749e-02],
        [ 2.4921e-02, -1.5488e-02, -1.4563e-02,  ..., -3.3916e-02,
         -3.5218e-02,  4.6975e-03],
        ...,
        [-6.3645e-04,  1.0396e-02, -1.9093e-02,  ...,  2.0065e-02,
          8.5642e-03,  2.7956e-02],
        [-3.0655e-02,  2.6850e-02, -2.5427e-02,  ..., -1.5808e-02,
          3.0509e-02, -2.7482e-02],
        [ 2.1631e-03, -5.4711e-03, -3.0814e-02,  ...,  3.2413e-02,
          1.8580e-02, -1.5256e-02]], device='cuda:0')), ('fc1.bias', tensor([ 8.2043e-02,  6.0930e-02,  9.9372e-03, -5.6765e-02,  6.8273e-02,
         1.3366e-02, -2.4556e-02,  5.1160e-02, -5.6142e-02, -1.2249e-02,
        -5.8979e-03,  8.6617e-02,  1.0421e-01,  3.0909e-02,  4.6933e-02,
         8.8124e-02,  7.6432e-02,  7.6576e-02,  6.7188e-02, -2.8336e-02,
         3.3370e